# Tutorial 6
We show how to train Double DQN for high frequency trading on Bitcoin.

## Step 1: Import Packages
Modify the system path and load the corresponding packages and functions 

In [ ]:
import os
import sys
from pathlib import Path
import warnings
warnings.filterwarnings("ignore")

ROOT = os.path.dirname(os.path.abspath("."))
sys.path.append(ROOT)

import torch
import argparse
import os.path as osp
from mmcv import Config
from trademaster.utils import replace_cfg_vals
from trademaster.nets.builder import build_net
from trademaster.environments.builder import build_environment
from trademaster.datasets.builder import build_dataset
from trademaster.agents.builder import build_agent
from trademaster.optimizers.builder import build_optimizer
from trademaster.losses.builder import build_loss
from trademaster.trainers.builder import build_trainer
from trademaster.transition.builder import build_transition
from trademaster.utils import plotfrom trademaster.utils import set_seedset_seed(2023)

## Step 2: Load Configs
Load default config from the file `configs/high_frequency_trading/high_frequency_trading_BTC_dqn_dqn_adam_mse.py`

In [ ]:
parser = argparse.ArgumentParser(description='Download Alpaca Datasets')
parser.add_argument("--config", default=osp.join(ROOT, "configs", "high_frequency_trading", "high_frequency_trading_BTC_dqn_dqn_adam_mse.py"),
                    help="download datasets config file path")
parser.add_argument("--task_name", type=str, default="train")

args, _ = parser.parse_known_args()
cfg = Config.fromfile(args.config)
task_name = args.task_name
cfg = replace_cfg_vals(cfg)

In [ ]:
cfg

## Step 3: Build Dataset

In [ ]:
dataset = build_dataset(cfg)

## Step 4: Build Reinforcement Learning Environments
Build environments based on cfg and predefined dataset



In [ ]:
valid_environment = build_environment(cfg, default_args=dict(dataset=dataset, task="valid"))
test_environment = build_environment(cfg, default_args=dict(dataset=dataset, task="test"))
cfg.environment = cfg.train_environment
train_environment = build_environment(cfg, default_args=dict(dataset=dataset, task="train"))

In [ ]:
train_environment.df.head()

## Step 5: Build Net 
Update information about the state and action dimension. Create networks and optimizer for Double DQN


In [ ]:
action_dim = train_environment.action_dim
state_dim = train_environment.state_dim

cfg.act.update(dict(action_dim=action_dim, state_dim=state_dim))
act = build_net(cfg.act)
act_optimizer = build_optimizer(cfg, default_args=dict(params=act.parameters()))
if cfg.cri:
    cfg.cri.update(dict(action_dim=action_dim, state_dim=state_dim))
    cri = build_net(cfg.cri)
    cri_optimizer = build_optimizer(cfg, default_args=dict(params=cri.parameters()))
else:
    cri = None
    cri_optimizer = None

## Step 6: Build Loss

In [ ]:
criterion = build_loss(cfg)

## Step 7: Build Agent

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
agent = build_agent(cfg,default_args=dict(action_dim=action_dim,state_dim=state_dim,act=act,cri=cri,act_optimizer=act_optimizer,cri_optimizer=cri_optimizer,criterion=criterion,device=device))


## Step 8: Build Trainer
Build trainer from config and create work directionary to save the result, model and config

In [ ]:

trainer = build_trainer(cfg, default_args=dict(train_environment=train_environment,valid_environment=valid_environment,test_environment=test_environment,agent=agent,device=device))

cfg.dump(osp.join(ROOT, cfg.work_dir, osp.basename(args.config)))

## Step 9: RL Agent Training
Train the Double DQN agent based on the config and save results in workdir

In [ ]:
trainer.train_and_valid()

## Step 10: RL Agent Testing

In [ ]:
trainer.test();

In [ ]:
plot(trainer.test_environment.save_asset_memoey(),alg="DDQN")